In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
cd /content/gdrive/My\ Drive/Data\ Sets

/content/gdrive/My Drive/Data Sets


In [0]:
!pip3 install sk-video

     |████████████████████████████████| 2.3MB 3.5MB/s 


In [0]:
import skvideo.io
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from glob import glob
import skimage
import keras
import keras.models
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [0]:
base_path = "ucfTrainTestlist/"
f = open(base_path + "trainlist01.txt", "r")
temp = f.read()
train = pd.DataFrame()
train['video_name'] = temp.split('\n')
train = train[:-1]

,video_name
0,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1
1,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c02.avi 1
2,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03.avi 1
3,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c04.avi 1
4,ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c05.avi 1


In [0]:
train_video_tag = []
for i in range(train.shape[0]):
    train_video_tag.append(train['video_name'][i].split('/')[0])
train['tag'] = train_video_tag
train_image_nm = []
train_class = []
train_image = []
default_size = (224, 224, 3)
print(train['video_name'][0])

ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c01.avi 1


In [0]:
for i in tqdm(range(train.shape[0])):
    videoFile = train['video_name'][i]
    path = 'UCF-101/' + train['tag'][i] + '/' + videoFile.split(' ')[0].split('/')[1]
    z = skvideo.io.vreader(path, inputdict={'-r' : 0.25})
    cnt = 0
    for i in z:
        cnt += 1
        train_image.append(skimage.transform.resize(np.squeeze(i[0]), default_size))
        train_image_nm.append(videoFile.split(' ')[0].split('/')[1] + ' ' + str(cnt))
        train_class.append(videoFile.split(' ')[0].split('/')[1].split('_')[1])

 12%|█▏        | 1165/9537 [11:47<1:52:24,  1.24it/s]

In [0]:
train_nw = pd.DataFrame()
train_nw['image'] = train_image_nm
train_nw['class'] = train_class
train_image = np.array(train_image)

In [0]:
y = train_nw
X_train, X_test, y_train, y_test = train_test_split(train_image, y, random_state=100, stratify = y, test_size=0.2)
y_train_name = y_train['image']
y_test_name = y_test['image']
y_train = pd.get_dummies(y_train['class'])
y_test = pd.get_dummies(y_test['class'])

In [0]:
init_model = InceptionResNetV2(include_top=False, weights='imagenet')
X_train = init_model.predict(X_train)
X_test = init_model.predict(X_test)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1]*X_train.shape[2]*X_train.shape[3])
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1]*X_test.shape[2]*X_test.shape[3])
mx_train = X_train.max()
X_train /= mx_train
X_test /= mx_train
print(X_train.shape, X_test.shape)

In [0]:
cnt = 0
v_train = []
v_test = []
vy_train = []
vy_test = []
while cnt < X_train.shape[0]:
    cnt_in = cnt
    vy_train.append(y_train[cnt])
    name_vid = y_train_name[cnt].split(' ')
    while cnt < X_train.shape[0] and y_train_name[cnt].split(' ') == name_vid:
        cnt += 1
    if cnt - cnt_in >= 3: v_train.append(X_train[cnt_in: cnt_in + 3])
    else:
        v_train.append(X_train[cnt_in: cnt] + X_train[cnt - 3 + cnt - cnt_in :cnt])

In [0]:
cnt = 0
while cnt < X_test.shape[0]:
    cnt_in = cnt
    vy_test.append(y_test[cnt])
    name_vid = y_test_name[cnt].split(' ')
    while cnt < X_test.shape[0] and y_test_name[cnt].split(' ') == name_vid:
        cnt += 1
    if cnt - cnt_in >= 3: v_test.append(X_test[cnt_in: cnt_in + 3])
    else:
        v_test.append(X_test[cnt_in: cnt] + X_test[cnt - 3 + cnt - cnt_in :cnt])

In [0]:
model = keras.models.Sequential([
    Conv3D(512, activation='relu', input_dim=X_train.shape[1]),
    Dropout(0.4),
    Conv3D(256, activation='relu'),
    Dropout(0.4),
    Conv3D(128, activation='relu'),
    Dropout(0.4),
    Conv3D(101, activation='softmax'),
])

In [0]:
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weights2.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
model.fit(X_train, y_train, epochs=150, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=256)